In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# classifier models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# modules to handle data
import pandas as pd
import numpy as np
import warnings

In [3]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
Passenger_id =test.PassengerId
titanic = pd.concat([train,test])
titanic.head()

train_idx = len(train)
test_idx = len(titanic)-len(test)

In [4]:
titanic.dtypes.sort_values()

PassengerId      int64
Pclass           int64
SibSp            int64
Parch            int64
Survived       float64
Age            float64
Fare           float64
Name            object
Sex             object
Ticket          object
Cabin           object
Embarked        object
dtype: object

In [5]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [7]:
titanic['Title'] = titanic.Name.apply(lambda x:x.split(',')[1].split('.')[0].strip())


In [8]:
normal_ranks = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}

titanic['Title'] = titanic.Title.map(normal_ranks)
print(titanic.Title.value_counts())

Title
Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: count, dtype: int64


In [9]:
grouped = titanic.groupby(['Pclass','Sex','Title'])
grouped.Age.median()

Pclass  Sex     Title  
1       female  Miss       30.0
                Mrs        45.0
                Officer    49.0
                Royalty    39.0
        male    Master      6.0
                Mr         41.5
                Officer    52.0
                Royalty    40.0
2       female  Miss       20.0
                Mrs        30.0
        male    Master      2.0
                Mr         30.0
                Officer    41.5
3       female  Miss       18.0
                Mrs        31.0
        male    Master      6.0
                Mr         26.0
Name: Age, dtype: float64

In [10]:
titanic['Age'] = grouped.Age.transform(lambda x: x.fillna(x.median()))
titanic['Cabin'] = titanic['Cabin'].fillna('U')
most_embarked = titanic.Embarked.value_counts().index[0]
titanic.Embarked = titanic.Embarked.fillna(most_embarked)
titanic.Fare = titanic.Fare.fillna(titanic.Fare.median())

titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1309 non-null   float64
 10  Cabin        1309 non-null   object 
 11  Embarked     1309 non-null   object 
 12  Title        1309 non-null   object 
dtypes: float64(3), int64(4), object(6)
memory usage: 143.2+ KB


In [11]:
titanic['Cabin'] = titanic['Cabin'].map(lambda x:x[0])
titanic["Fam"] = titanic['Parch'] + titanic['SibSp'] +1
titanic.Sex = titanic.Sex.map({"male": 0, "female":1})

pclass_dummies = pd.get_dummies(titanic.Pclass, prefix="Pclass")
title_dummies = pd.get_dummies(titanic.Title, prefix="Title")
cabin_dummies = pd.get_dummies(titanic.Cabin, prefix="Cabin")
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix="Embarked")

titanic_dummies = pd.concat([titanic, pclass_dummies, title_dummies, cabin_dummies, embarked_dummies], axis=1)

titanic_dummies.drop(['Pclass', 'Title', 'Cabin', 'Embarked', 'Name', 'Ticket'], axis=1, inplace=True)

titanic_dummies.head()

,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,Fam,Pclass_1,Pclass_2,...,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S
0,1,0.0,0,22.0,1,0,7.2500,2,False,False,...,False,False,False,False,False,False,True,False,False,True
1,2,1.0,1,38.0,1,0,71.2833,2,True,False,...,True,False,False,False,False,False,False,True,False,False
2,3,1.0,1,26.0,0,0,7.9250,1,False,False,...,False,False,False,False,False,False,True,False,False,True
3,4,1.0,1,35.0,1,0,53.1000,2,True,False,...,True,False,False,False,False,False,False,False,False,True
4,5,0.0,0,35.0,0,0,8.0500,1,False,False,...,False,False,False,False,False,False,True,False,False,True


In [12]:
train = titanic_dummies[:train_idx]
test = titanic_dummies[test_idx:]
train.Survived= train.Survived.astype(int)

/tmp/ipykernel_13/3710624305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Survived= train.Survived.astype(int)


In [13]:
X = train.drop('Survived', axis=1).values 
y = train.Survived.values

X_test = test.drop('Survived', axis=1).values

In [14]:
forrest_params = dict(     
    max_depth = [n for n in range(9, 14)],     
    min_samples_split = [n for n in range(4, 11)], 
    min_samples_leaf = [n for n in range(2, 5)],     
    n_estimators = [n for n in range(10, 60, 10)],
)

In [15]:
forrest = RandomForestClassifier()
forest_cv = GridSearchCV(estimator=forrest, param_grid=forrest_params, cv=5) 
forest_cv.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [9, 10, 11, 12, 13],
                         'min_samples_leaf': [2, 3, 4],
                         'min_samples_split': [4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [10, 20, 30, 40, 50]})

In [16]:
print("Best score: {}".format(forest_cv.best_score_))
print("Optimal params: {}".format(forest_cv.best_estimator_))

Best score: 0.8395016006528152
Optimal params: RandomForestClassifier(max_depth=12, min_samples_leaf=3, min_samples_split=5,
                       n_estimators=50)


In [17]:
forrest_pred = forest_cv.predict(X_test)

In [18]:
kaggle = pd.DataFrame({'PassengerId': Passenger_id, 'Survived': forrest_pred})
kaggle.to_csv('/kaggle/working//titanic_pred.csv', index=False)